# Analisi e Filtri sui Goal

Questo notebook mostra come caricare i dati dei match dal database e applicare filtri basati sui goal segnati.
Nota: Per analizzare i risultati del primo tempo, assicurati di aver aggiornato il database con le colonne `home_score_ht` e `away_score_ht`.

In [3]:
import sys
import os
import pandas as pd
import psycopg2
import requests
import importlib
import warnings
warnings.filterwarnings('ignore')

# Aggiungiamo la root del progetto al path per importare i moduli
from modules.config import DB_CONFIG
import modules.analysis
importlib.reload(modules.analysis)
from modules.analysis import get_matches, get_stats_by_period, get_matches_by_partial_score, get_matches_by_ht_score, get_len_table

# Carichiamo i dati iniziali
df = get_matches()

In [4]:
# Preprocessing dei dati
# Convertiamo i punteggi in numeri (gestendo eventuali 'N/A' o errori)
df['home_score'] = pd.to_numeric(df['home_score'], errors='coerce')
df['away_score'] = pd.to_numeric(df['away_score'], errors='coerce')

# HT Scores (potrebbero essere NaN per vecchi match)
df['home_score_ht'] = pd.to_numeric(df['home_score_ht'], errors='coerce')
df['away_score_ht'] = pd.to_numeric(df['away_score_ht'], errors='coerce')

# Rimuoviamo righe dove i punteggi finali sono NaN
df = df.dropna(subset=['home_score', 'away_score'])

# Calcoliamo i goal totali
df['total_goals'] = df['home_score'] + df['away_score']

df.head()

,id,home_team,away_team,home_score,away_score,home_score_ht,away_score_ht,tournament,start_timestamp,total_goals
0,12436498,Chelsea,Aston Villa,3.0,0.0,NaN,NaN,Premier League,1733059800,3.0
1,12436505,Manchester United,Everton,4.0,0.0,NaN,NaN,Premier League,1733059800,4.0
2,12436507,Tottenham Hotspur,Fulham,1.0,1.0,NaN,NaN,Premier League,1733059800,2.0
3,12436504,Liverpool,Manchester City,2.0,0.0,NaN,NaN,Premier League,1733068800,2.0
4,12504665,Udinese,Genoa,0.0,2.0,NaN,NaN,Serie A,1733052600,2.0


## Esempio di Filtro per Goal

Qui sotto mostriamo come filtrare il DataFrame per trovare i match che soddisfano determinati criteri di punteggio.

In [5]:
# ESEMPIO 1: Filtro per "Over 2.5" (più di 2 goal totali)
over_25_matches = df[df['total_goals'] > 2.5]

# ESEMPIO 2: Filtro per "Goal/Goal" (entrambe le squadre segnano)
goal_goal_matches = df[(df['home_score'] > 0) & (df['away_score'] > 0)]

# ESEMPIO 3: Filtro combinato (Over 2.5 AND Goal/Goal)
combined_filter = df[(df['total_goals'] > 2.5) & (df['home_score'] > 0) & (df['away_score'] > 0)]

print(f"Totale match validi: {len(df)}")
print(f"Match Over 2.5: {len(over_25_matches)} ({len(over_25_matches)/len(df)*100:.1f}%)")
print(f"Match Goal/Goal: {len(goal_goal_matches)} ({len(goal_goal_matches)/len(df)*100:.1f}%)")

Totale match validi: 980
Match Over 2.5: 504 (51.4%)
Match Goal/Goal: 503 (51.3%)


## Funzioni di Utility per Statistiche Avanzate

## Analisi Primo Tempo (Basata su Statistiche)

Utilizziamo la tabella `match_statistics_column` per trovare partite con caratteristiche specifiche al primo tempo (es. 0 tiri in porta), superando il limite dei dati mancanti nel DB principale.

In [6]:
# Carichiamo i tiri in porta del primo tempo (1ST)
try:
    df_tech_ht = get_stats_by_period(['Shots on target'], period='1ST')
    
    # Uniamo al dataframe principale per avere i nomi delle squadre e i risultati finali
    df_merged_ht = df.merge(df_tech_ht, left_on='id', right_on='match_id', how='inner')

    # FILTRO: Partite con 0 Tiri in porta totali nel primo tempo
    # (Un ottimo proxy per match anestetizzati o 0-0 tecnici quando il punteggio HT manca)
    stati_anestetizzati = df_merged_ht[
        (df_merged_ht['shots_on_target_homevalue'] == 0) & 
        (df_merged_ht['shots_on_target_awayvalue'] == 0)
    ]

    print(f"Match con statistiche HT disponibili: {len(df_merged_ht)}")
    print(f"Partite con 0 tiri in porta (HT): {len(stati_anestetizzati)}")

    if not stati_anestetizzati.empty:
        display(stati_anestetizzati[['home_team', 'away_team', 'home_score', 'away_score', 'total_goals']].head(10))
        print(f"\nMedia goal finali per queste partite: {stati_anestetizzati['total_goals'].mean():.2f}")
except Exception as e:
    print(f"Errore: {e}")

Match con statistiche HT disponibili: 402
Partite con 0 tiri in porta (HT): 11


,home_team,away_team,home_score,away_score,total_goals
7,Lecce,Juventus,1.0,1.0,2.0
85,Lyngby,Sønderjyske Fodbold,0.0,2.0,2.0
104,Strømsgodset,Molde FK,1.0,0.0,1.0
143,Nottingham Forest,Arsenal,0.0,0.0,0.0
180,Santa Clara,Famalicão,0.0,1.0,1.0
234,Albacete Balompié,Cádiz,1.0,0.0,1.0
271,Esenler Erokspor,Vanspor FK,0.0,0.0,0.0
272,Erzurumspor FK,Amed Sportif Faaliyetler,2.0,0.0,2.0
334,Al-Sailiya,Qatar SC,2.0,1.0,3.0
394,Atlético Mineiro,Tombense,0.0,0.0,0.0



Media goal finali per queste partite: 1.64


## Analisi Avanzata xG HT

In questa sezione analizziamo la pericolosità (xG) del primo tempo indipendentemente dal punteggio.

In [7]:
try:
    # Carichiamo xG del primo tempo
    df_xg_ht = get_stats_by_period(['Expected goals'], period='1ST')
    
    # Uniamo al dataframe principale
    df_advanced = df.merge(df_xg_ht, left_on='id', right_on='match_id', how='inner')
    
    # Calcoliamo xG totale del primo tempo
    if 'expected_goals_homevalue' in df_advanced.columns:
        df_advanced['total_xg_ht'] = df_advanced['expected_goals_homevalue'] + df_advanced['expected_goals_awayvalue']
    
    print(f"Match con statistiche xG caricate: {len(df_advanced)}")
    
    # ANALISI: Partite con xG HT totale > 1.2
    pericolose_ht = df_advanced[df_advanced['total_xg_ht'] > 1.2]
    
    print(f"Partite con xG HT totale > 1.2: {len(pericolose_ht)}")
    if not pericolose_ht.empty:
        display(pericolose_ht[['home_team', 'away_team', 'total_xg_ht', 'home_score', 'away_score', 'total_goals']])
        
        # Statistica: quante di queste finiscono con almeno 2 goal totali?
        over15 = len(pericolose_ht[pericolose_ht['total_goals'] >= 1.5])
        print(f"\nPercentuale Over 1.5 finale per queste partite: {over15/len(pericolose_ht)*100:.1f}%")
except Exception as e:
    print(f"Errore: {e}")

Match con statistiche xG caricate: 121
Partite con xG HT totale > 1.2: 57


,home_team,away_team,total_xg_ht,home_score,away_score,total_goals
0,Chelsea,Aston Villa,1.67,3.0,0.0,3.0
2,Tottenham Hotspur,Fulham,1.24,1.0,1.0,2.0
3,Liverpool,Manchester City,1.83,2.0,0.0,2.0
4,Udinese,Genoa,1.47,0.0,2.0,2.0
7,Lecce,Juventus,1.21,1.0,1.0,2.0
8,Villarreal,Girona FC,1.67,2.0,2.0,4.0
12,1. FSV Mainz 05,TSG Hoffenheim,2.00,2.0,0.0,2.0
14,Montpellier,Lille,1.39,2.0,2.0,4.0
15,Le Havre,Angers,1.83,0.0,1.0,1.0
16,Olympique Lyonnais,Nice,2.47,4.0,1.0,5.0



Percentuale Over 1.5 finale per queste partite: 87.7%


## Ricerca Match per Risultato Primo Tempo (HT)

Utilizziamo le colonne `home_score_ht` e `away_score_ht` della tabella `matches` per trovare rapidamente i risultati a fine primo tempo.

In [8]:
# ESEMPIO: Partite che erano sull'1-0 al termine del primo tempo
HT_HOME = 0
HT_AWAY = 0

print(f"Ricerca match con punteggio HT {HT_HOME}-{HT_AWAY}...")
res_ht = get_matches_by_ht_score(HT_HOME, HT_AWAY)
len_table = get_len_table("matches")

if not res_ht.empty:
    print(f"Trovati {len(res_ht)} match su {len_table} totali. ({len(res_ht)/len_table*100:.2f}%)")
    display(res_ht.head(20))
else:
    print("Nessun match trovato con questo punteggio HT.")

Ricerca match con punteggio HT 0-0...
Trovati 166 match su 1007 totali. (16.48%)


,id,home_team,away_team,home_score_ht,away_score_ht,final_h,final_a
0,14058091,Egypt,Nigeria,0,0,2,4
1,14058089,Senegal,Morocco,0,0,1,0
2,14025079,Manchester United,Manchester City,0,0,2,0
3,14025073,Leeds United,Fulham,0,0,1,0
4,14025082,Nottingham Forest,Arsenal,0,0,0,0
5,14025092,Wolverhampton,Newcastle United,0,0,0,0
6,14025064,Aston Villa,Everton,0,0,0,1
7,13981625,Cagliari,Juventus,0,0,1,0
8,13981627,Parma,Genoa,0,0,0,0
9,13981628,Milan,Lecce,0,0,1,0


### Filtro Avanzato: Match con Pochi Tiri nel Primo Tempo

Analizziamo quali tra i match trovati sopra avevano un numero di tiri totali (Home + Away) compreso tra 0 e 5 nel primo tempo.

In [23]:
STAMPA_PARTITE = False
TIPO_STATISTICA = "Shots on target" # "Total shots", "Shots on target", "Hit woodwork", "Shots off target", "Blocked shots", "Shots inside box", "Shots outside box"

# Range
RANGE_1 = [0, 5]
RANGE_2 = [5, 10]
RANGE_3 = [10, 15]

try:
    # 1. Recuperiamo la statistica scelta
    df_stats_ht = get_stats_by_period([TIPO_STATISTICA], period='1ST')
    if df_stats_ht.empty: 
        print(f"Nessun dato per {tipo_statistica}")
        
    # Pulizia nomi colonne dinamica
    stat_clean = tipo_statistica.lower().replace(' ', '_')
    stat_col_h = f"{stat_clean}_homevalue"
    stat_col_a = f"{stat_clean}_awayvalue"
    
    # 2. Uniamo con i risultati HT
    res_ht_advanced = res_ht.merge(df_stats_ht, left_on='id', right_on='match_id', how='inner')
    
    # 3. Calcoliamo il totale per il range
    res_ht_advanced['total_val_ht'] = res_ht_advanced[stat_col_h] + res_ht_advanced[stat_col_a]
    
    # 4. Filtriamo per range
    result_range_1 = res_ht_advanced[(res_ht_advanced['total_val_ht'] >= RANGE_1[0]) & (res_ht_advanced['total_val_ht'] <= RANGE_1[1])]
    result_range_2 = res_ht_advanced[(res_ht_advanced['total_val_ht'] > RANGE_2[0]) & (res_ht_advanced['total_val_ht'] <= RANGE_2[1])]
    result_range_3 = res_ht_advanced[(res_ht_advanced['total_val_ht'] > RANGE_3[0]) & (res_ht_advanced['total_val_ht'] <= RANGE_3[1])]
    
    print(f"Analisi su {tipo_statistica} (Match HT {HT_HOME}-{HT_AWAY}):")
    
    for label, df_f in [(f"{RANGE_1[0]}-{RANGE_1[1]}", result_range_1), (f"{RANGE_2[0]}-{RANGE_2[1]}", result_range_2), (f"{RANGE_3[0]}-{RANGE_3[1]}", result_range_3)]:
        print(f"\n--- Range {tipo_statistica}: {label} ---")
        print(f"Match trovati: {len(df_f)} ({len(df_f)/len(res_ht)*100:.1f}%) health: {len(df_f)/len(res_ht_advanced)*100:.1f}% dei match con statistiche")
        if not df_f.empty:
            if STAMPA_PARTITE:    
                display(df_f[['home_team', 'away_team', 'total_val_ht', 'final_h', 'final_a']].head(10))
            if HT_HOME == 0 and HT_AWAY == 0:
                over05 = len(df_f[(df_f['final_h'] + df_f['final_a']) > 0])
                print(f"Percentuale Over 0.5 finale: {over05/len(df_f)*100:.1f}%")
                
except Exception as e:
    print(f"Errore: {e}")

Analisi su Shots on target (Match HT 0-0):

--- Range Shots on target: 0-5 ---
Match trovati: 92 (55.4%) health: 92.9% dei match con statistiche
Percentuale Over 0.5 finale: 73.9%

--- Range Shots on target: 5-10 ---
Match trovati: 7 (4.2%) health: 7.1% dei match con statistiche
Percentuale Over 0.5 finale: 85.7%

--- Range Shots on target: 10-15 ---
Match trovati: 0 (0.0%) health: 0.0% dei match con statistiche


## Ricerca Match per Risultato Parziale

Utilizziamo la tabella `match_incidents_column` per trovare le partite che avevano un punteggio specifico ad un minuto specifico.

In [ ]:
# ESEMPIO: Partite che erano sull'1-1 al minuto 75
TARGET_MIN = 45
T_HOME = 0
T_AWAY = 0

print(f"Ricerca match con punteggio {T_HOME}-{T_AWAY} al minuto {TARGET_MIN}...")
res = get_matches_by_partial_score(T_HOME, T_AWAY, TARGET_MIN)

if not res.empty:
    print(f"Trovati {len(res)} match.")
    display(res.head(20))
else:
    print("Nessun match trovato con questi criteri.")

Ricerca match con punteggio 0-0 al minuto 45...
Trovati 727 match.


c:\Users\nicol\Desktop\Progetti\pySofa\scripts\analyze_score_frequency\modules\analysis.py:61: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  query = """


,id,home_team,away_team,score_h,score_a,final_h,final_a
0,12436498,Chelsea,Aston Villa,0,0,3.0,0.0
1,12436505,Manchester United,Everton,0,0,4.0,0.0
2,12436507,Tottenham Hotspur,Fulham,0,0,1.0,1.0
3,12436504,Liverpool,Manchester City,0,0,2.0,0.0
4,12504665,Udinese,Genoa,0,0,0.0,2.0
5,12504666,Parma,Lazio,0,0,3.0,1.0
6,12504669,Torino,Napoli,0,0,0.0,1.0
7,12504674,Lecce,Juventus,0,0,1.0,1.0
8,12504673,Roma,Atalanta,0,0,0.0,2.0
9,12437727,Villarreal,Girona FC,0,0,2.0,2.0
